In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from datetime import timedelta

receivals = pd.read_csv('./Project_materials/data/kernel/receivals.csv')
purchase_orders = pd.read_csv('./Project_materials/data/kernel/purchase_orders.csv')
prediction_mapping = pd.read_csv('./Project_materials/data/prediction_mapping.csv')
sample_submission = pd.read_csv('./Project_materials/data/sample_submission.csv')

# Convert dates
receivals['date_arrival'] = pd.to_datetime(receivals['date_arrival'], utc=True).dt.tz_localize(None)
purchase_orders['delivery_date'] = pd.to_datetime(purchase_orders['delivery_date'], utc=True).dt.tz_localize(None)
purchase_orders['created_date_time'] = pd.to_datetime(purchase_orders['created_date_time'], utc=True).dt.tz_localize(None)
prediction_mapping['forecast_start_date'] = pd.to_datetime(prediction_mapping['forecast_start_date'])
prediction_mapping['forecast_end_date'] = pd.to_datetime(prediction_mapping['forecast_end_date'])

print("="*80)
print("STEP 8: ADD TIMING FEATURES TO BEST MODEL")
print("="*80)

# ============================================================================
# DATA CLEANING
# ============================================================================
print("\n[1] DATA CLEANING")
receivals = receivals[receivals['net_weight'] > 0]
receivals = receivals[receivals['rm_id'].notna()]
receivals = receivals.sort_values('date_arrival')
print(f"Clean receivals: {len(receivals)}")

# ============================================================================
# PRE-COMPUTE TIMING STATISTICS FOR EACH RM_ID
# ============================================================================
print("\n[2] PRE-COMPUTING TIMING STATISTICS")
print("-"*80)

timing_stats = {}

for rm_id in receivals['rm_id'].unique():
    rm_hist = receivals[receivals['rm_id'] == rm_id].sort_values('date_arrival')
    
    if len(rm_hist) >= 2:
        # Calculate inter-arrival intervals
        intervals = rm_hist['date_arrival'].diff().dt.days.dropna()
        
        if len(intervals) > 0:
            avg_days_between = intervals.mean()
            std_days_between = intervals.std()
            cv_days_between = std_days_between / avg_days_between if avg_days_between > 0 else 999
        else:
            avg_days_between = 0
            cv_days_between = 999
    else:
        # Only 1 or 0 deliveries - can't compute intervals
        avg_days_between = 0
        cv_days_between = 999
    
    timing_stats[rm_id] = {
        'avg_days_between': avg_days_between,
        'cv_days_between': cv_days_between
    }

print(f"Computed timing stats for {len(timing_stats)} rm_ids")
print(f"RM_IDs with computable intervals: {sum(1 for v in timing_stats.values() if v['avg_days_between'] > 0)}")

# ============================================================================
# TRAINING DATA GENERATION (BEST MODEL FEATURES + TIMING FEATURES)
# ============================================================================
print("\n[3] CREATING TRAINING DATA WITH TIMING FEATURES")
print("-"*80)

train_dates = pd.date_range(start='2024-01-01', end='2024-11-30', freq='MS')
forecast_horizons = [7, 30, 60, 90, 150]

print(f"Using {len(train_dates)} training dates x {len(forecast_horizons)} horizons")

training_data = []
active_rm_ids = receivals[receivals['date_arrival'] >= '2024-01-01']['rm_id'].unique()
print(f"Active rm_ids in 2024: {len(active_rm_ids)}")

for i, train_date in enumerate(train_dates):
    print(f"Processing date {i+1}/{len(train_dates)}: {train_date.date()}...")
    
    for rm_id in active_rm_ids:
        hist = receivals[
            (receivals['rm_id'] == rm_id) &
            (receivals['date_arrival'] < train_date)
        ]
        
        if len(hist) == 0:
            continue
        
        cutoff_365 = train_date - timedelta(days=365)
        cutoff_180 = train_date - timedelta(days=180)
        cutoff_90 = train_date - timedelta(days=90)
        cutoff_30 = train_date - timedelta(days=30)
        
        recent_365 = hist[hist['date_arrival'] >= cutoff_365]
        recent_180 = hist[hist['date_arrival'] >= cutoff_180]
        recent_90 = hist[hist['date_arrival'] >= cutoff_90]
        recent_30 = hist[hist['date_arrival'] >= cutoff_30]
        
        # Basic aggregations (ORIGINAL FEATURES)
        if len(recent_365) > 0:
            total_365 = recent_365['net_weight'].sum()
            count_365 = len(recent_365)
            days_since = (train_date - recent_365['date_arrival'].max()).days
        else:
            total_365 = count_365 = days_since = 0
        
        if len(recent_180) > 0:
            total_180 = recent_180['net_weight'].sum()
            count_180 = len(recent_180)
        else:
            total_180 = count_180 = 0
        
        if len(recent_90) > 0:
            total_90 = recent_90['net_weight'].sum()
            count_90 = len(recent_90)
        else:
            total_90 = count_90 = 0
        
        if len(recent_30) > 0:
            total_30 = recent_30['net_weight'].sum()
            count_30 = len(recent_30)
            rate_30 = total_30 / 30
        else:
            total_30 = count_30 = rate_30 = 0
        
        # Rates
        rate_90 = total_90 / 90 if total_90 > 0 else 0
        
        # Recency-weighted sum
        if len(recent_90) > 0:
            days_ago = (train_date - recent_90['date_arrival']).dt.days
            weights = 1.0 / (days_ago + 1)
            recency_weighted = (recent_90['net_weight'] * weights).sum()
        else:
            recency_weighted = 0
        
        # Active days ratio
        if len(recent_90) > 0:
            active_days_90 = recent_90['date_arrival'].dt.date.nunique()
            active_ratio_90 = active_days_90 / 90
        else:
            active_ratio_90 = 0
        
        # NEW: TIMING FEATURES
        # Get pre-computed timing stats
        timing = timing_stats.get(rm_id, {'avg_days_between': 0, 'cv_days_between': 999})
        avg_days_between = timing['avg_days_between']
        cv_days_between = timing['cv_days_between']
        
        for horizon in forecast_horizons:
            forecast_end = train_date + timedelta(days=horizon)
            
            actual = receivals[
                (receivals['rm_id'] == rm_id) &
                (receivals['date_arrival'] >= train_date) &
                (receivals['date_arrival'] <= forecast_end)
            ]
            target = actual['net_weight'].sum()
            
            # NEW: Calculate horizon-dependent timing features
            if avg_days_between > 0:
                expected_deliveries_in_horizon = horizon / avg_days_between
                delivery_overdue_ratio = days_since / avg_days_between if days_since > 0 else 0
            else:
                expected_deliveries_in_horizon = 0
                delivery_overdue_ratio = 999  # No history of intervals
            
            training_data.append({
                # Original features
                'rm_id': rm_id,
                'train_date': train_date,
                'forecast_horizon': horizon,
                'total_weight_365d': total_365,
                'count_365d': count_365,
                'days_since_last': days_since,
                'total_weight_90d': total_90,
                'count_90d': count_90,
                'rate_90': rate_90,
                'total_weight_180d': total_180,
                'count_180d': count_180,
                'total_30': total_30,
                'count_30': count_30,
                'rate_30': rate_30,
                'recency_weighted': recency_weighted,
                'active_ratio_90': active_ratio_90,
                # NEW: Timing features
                'avg_days_between': avg_days_between,
                'cv_days_between': cv_days_between,
                'expected_deliveries_in_horizon': expected_deliveries_in_horizon,
                'delivery_overdue_ratio': delivery_overdue_ratio,
                'target': target
            })

print(f"\nGenerated {len(training_data)} training samples")
train_df = pd.DataFrame(training_data)

print(f"Samples with target > 0: {(train_df['target'] > 0).sum()} ({(train_df['target'] > 0).sum() / len(train_df) * 100:.1f}%)")

# ============================================================================
# TIME-BASED TRAIN/VAL SPLIT
# ============================================================================
print("\n[4] TIME-BASED TRAIN/VAL SPLIT")
print("-"*80)

split_date = pd.to_datetime('2024-09-01')

train_mask = train_df['train_date'] < split_date
val_mask = train_df['train_date'] >= split_date

feature_cols = [c for c in train_df.columns if c not in ['target', 'train_date']]

X_train = train_df[train_mask][feature_cols]
y_train = train_df[train_mask]['target']
X_val = train_df[val_mask][feature_cols]
y_val = train_df[val_mask]['target']

print(f"Training samples (before {split_date.date()}): {len(X_train)}")
print(f"Validation samples (>= {split_date.date()}): {len(X_val)}")

train_mean = y_train.mean()
val_mean = y_val.mean()
print(f"\nTarget statistics:")
print(f"  Training mean: {train_mean:,.0f} kg")
print(f"  Validation mean: {val_mean:,.0f} kg")
print(f"  Difference: {((val_mean - train_mean) / train_mean * 100):+.1f}%")

print(f"\nNumber of features: {len(feature_cols)}")
print("Features:", feature_cols)

# ============================================================================
# TRAIN LightGBM MODELS (IDENTICAL TO BEST MODEL)
# ============================================================================
print("\n[5] TRAINING LightGBM MODELS")
print("-"*80)

print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")

# Classifier
y_train_bin = (y_train > 0).astype(int)
y_val_bin = (y_val > 0).astype(int)

clf = lgb.LGBMClassifier(
    n_estimators=400,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight='balanced',
    random_state=42,
    verbose=-1
)
print("Training LightGBM Classifier...")
clf.fit(
    X_train, y_train_bin,
    eval_set=[(X_val, y_val_bin)],
    callbacks=[lgb.log_evaluation(period=100)]
)

# Regressor with alpha=0.10 (KEEPING BEST MODEL VALUE)
model = lgb.LGBMRegressor(
    objective='quantile',
    alpha=0.1,
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbose=-1
)

print(f"Training LightGBM Regressor (quantile={model.alpha})...")
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[lgb.log_evaluation(period=100)]
)

# ============================================================================
# FEATURE IMPORTANCE ANALYSIS
# ============================================================================
print("\n" + "="*80)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*80)

# Classifier importance
clf_importance = pd.DataFrame({
    'feature': feature_cols,
    'clf_importance': clf.feature_importances_,
    'clf_pct': clf.feature_importances_ / clf.feature_importances_.sum() * 100
}).sort_values('clf_importance', ascending=False)

print("\n[A] CLASSIFIER FEATURE IMPORTANCE (Top 15)")
print("-"*80)
for idx, row in clf_importance.head(15).iterrows():
    print(f"  {row['feature']:35s}: {row['clf_importance']:8.1f} ({row['clf_pct']:5.2f}%)")

# Regressor importance
reg_importance = pd.DataFrame({
    'feature': feature_cols,
    'reg_importance': model.feature_importances_,
    'reg_pct': model.feature_importances_ / model.feature_importances_.sum() * 100
}).sort_values('reg_importance', ascending=False)

print("\n[B] REGRESSOR FEATURE IMPORTANCE (Top 15)")
print("-"*80)
for idx, row in reg_importance.head(15).iterrows():
    print(f"  {row['feature']:35s}: {row['reg_importance']:8.1f} ({row['reg_pct']:5.2f}%)")

# Combined ranking
combined = pd.merge(
    clf_importance[['feature', 'clf_pct']],
    reg_importance[['feature', 'reg_pct']],
    on='feature'
)
combined['avg_pct'] = (combined['clf_pct'] + combined['reg_pct']) / 2
combined = combined.sort_values('avg_pct', ascending=False)

print("\n[C] COMBINED IMPORTANCE (Top 15)")
print("-"*80)
for idx, row in combined.head(15).iterrows():
    print(f"  {row['feature']:35s}: Clf={row['clf_pct']:5.2f}% | Reg={row['reg_pct']:5.2f}% | Avg={row['avg_pct']:5.2f}%")

# Identify low-importance features
low_importance = combined[combined['avg_pct'] < 1.0]
print("\n[D] LOW IMPORTANCE FEATURES (<1% avg)")
print("-"*80)
if len(low_importance) > 0:
    for idx, row in low_importance.iterrows():
        print(f"  {row['feature']:35s}: {row['avg_pct']:5.2f}%")
    print(f"\n⚠️  Consider removing {len(low_importance)} features in next iteration")
else:
    print("  All features have >1% importance ✓")

# Highlight NEW timing features
print("\n[E] NEW TIMING FEATURES PERFORMANCE")
print("-"*80)
timing_features = ['avg_days_between', 'cv_days_between', 'expected_deliveries_in_horizon', 'delivery_overdue_ratio']
for feat in timing_features:
    row = combined[combined['feature'] == feat].iloc[0]
    print(f"  {row['feature']:35s}: Clf={row['clf_pct']:5.2f}% | Reg={row['reg_pct']:5.2f}% | Avg={row['avg_pct']:5.2f}%")

# ============================================================================
# VALIDATION PERFORMANCE
# ============================================================================
print("\n" + "="*80)
print("VALIDATION PERFORMANCE")
print("="*80)

# Make predictions on validation set
val_reg_pred = np.maximum(0, model.predict(X_val))
val_clf_prob = clf.predict_proba(X_val)[:, 1]
val_combined_pred = val_reg_pred * val_clf_prob

# Calculate quantile loss
def quantile_loss_0_2(y_true, y_pred):
    error = y_true - y_pred
    return np.where(error >= 0, 0.2 * error, -0.8 * error)

quantile_loss = quantile_loss_0_2(y_val, val_combined_pred)
mean_quantile_loss = quantile_loss.mean()

print(f"Mean Quantile Loss (0.2): {float(mean_quantile_loss):,.2f}")
print(f"Median Quantile Loss: {float(np.median(quantile_loss)):,.2f}")
print(f"Mean Absolute Error: {float(np.abs(y_val - val_combined_pred).mean()):,.2f}")
print(f"\nMean Target: {float(y_val.mean()):,.2f}")
print(f"Mean Prediction: {float(val_combined_pred.mean()):,.2f}")
bias_pct = float((val_combined_pred.mean() - y_val.mean()) / y_val.mean() * 100)
print(f"Bias: {bias_pct:+.1f}%")

# ============================================================================
# MAKE PREDICTIONS FOR 2025
# ============================================================================
print("\n[6] MAKING PREDICTIONS FOR 2025")
print("-"*80)

forecast_start = pd.to_datetime('2025-01-01')

# Pre-compute features for all rm_ids (INCLUDING TIMING FEATURES)
rm_features = {}

for rm_id in prediction_mapping['rm_id'].unique():
    hist = receivals[
        (receivals['rm_id'] == rm_id) &
        (receivals['date_arrival'] < forecast_start)
    ]
    
    if len(hist) == 0:
        rm_features[rm_id] = {
            'total_weight_365d': 0,
            'count_365d': 0,
            'days_since_last': 999,
            'total_weight_90d': 0,
            'count_90d': 0,
            'rate_90': 0,
            'total_weight_180d': 0,
            'count_180d': 0,
            'total_30': 0,
            'count_30': 0,
            'rate_30': 0,
            'recency_weighted': 0,
            'active_ratio_90': 0,
            'avg_days_between': 0,
            'cv_days_between': 999,
        }
        continue
    
    cutoff_365 = forecast_start - timedelta(days=365)
    cutoff_180 = forecast_start - timedelta(days=180)
    cutoff_90 = forecast_start - timedelta(days=90)
    cutoff_30 = forecast_start - timedelta(days=30)
    
    recent_365 = hist[hist['date_arrival'] >= cutoff_365]
    recent_180 = hist[hist['date_arrival'] >= cutoff_180]
    recent_90 = hist[hist['date_arrival'] >= cutoff_90]
    recent_30 = hist[hist['date_arrival'] >= cutoff_30]
    
    if len(recent_365) > 0:
        total_365 = recent_365['net_weight'].sum()
        count_365 = len(recent_365)
        days_since = (forecast_start - recent_365['date_arrival'].max()).days
    else:
        total_365 = count_365 = days_since = 0
    
    if len(recent_180) > 0:
        total_180 = recent_180['net_weight'].sum()
        count_180 = len(recent_180)
    else:
        total_180 = count_180 = 0
    
    if len(recent_90) > 0:
        total_90 = recent_90['net_weight'].sum()
        count_90 = len(recent_90)
        rate_90 = total_90 / 90
    else:
        total_90 = count_90 = rate_90 = 0
    
    if len(recent_30) > 0:
        total_30 = recent_30['net_weight'].sum()
        count_30 = len(recent_30)
        rate_30 = total_30 / 30
    else:
        total_30 = count_30 = rate_30 = 0
    
    # Recency-weighted
    if len(recent_90) > 0:
        days_ago = (forecast_start - recent_90['date_arrival']).dt.days
        weights = 1.0 / (days_ago + 1)
        recency_weighted = (recent_90['net_weight'] * weights).sum()
    else:
        recency_weighted = 0
    
    # Active ratio
    if len(recent_90) > 0:
        active_days_90 = recent_90['date_arrival'].dt.date.nunique()
        active_ratio_90 = active_days_90 / 90
    else:
        active_ratio_90 = 0
    
    # NEW: Get timing stats
    timing = timing_stats.get(rm_id, {'avg_days_between': 0, 'cv_days_between': 999})
    
    rm_features[rm_id] = {
        'total_weight_365d': total_365,
        'count_365d': count_365,
        'days_since_last': days_since,
        'total_weight_90d': total_90,
        'count_90d': count_90,
        'rate_90': rate_90,
        'total_weight_180d': total_180,
        'count_180d': count_180,
        'total_30': total_30,
        'count_30': count_30,
        'rate_30': rate_30,
        'recency_weighted': recency_weighted,
        'active_ratio_90': active_ratio_90,
        'avg_days_between': timing['avg_days_between'],
        'cv_days_between': timing['cv_days_between'],
    }

print(f"Pre-computed features for {len(rm_features)} rm_ids")

# Make predictions
predictions = []

for idx, row in prediction_mapping.iterrows():
    rm_id = row['rm_id']
    forecast_end = row['forecast_end_date']
    horizon = (forecast_end - forecast_start).days + 1
    
    feat = rm_features[rm_id]
    
    # Calculate horizon-dependent timing features
    avg_days_between = feat['avg_days_between']
    days_since = feat['days_since_last']
    
    if avg_days_between > 0:
        expected_deliveries_in_horizon = horizon / avg_days_between
        delivery_overdue_ratio = days_since / avg_days_between if days_since > 0 else 0
    else:
        expected_deliveries_in_horizon = 0
        delivery_overdue_ratio = 999
    
    feature_dict = {
        'rm_id': rm_id,
        'forecast_horizon': horizon,
        'total_weight_365d': feat['total_weight_365d'],
        'count_365d': feat['count_365d'],
        'days_since_last': feat['days_since_last'],
        'total_weight_90d': feat['total_weight_90d'],
        'count_90d': feat['count_90d'],
        'rate_90': feat['rate_90'],
        'total_weight_180d': feat['total_weight_180d'],
        'count_180d': feat['count_180d'],
        'total_30': feat['total_30'],
        'count_30': feat['count_30'],
        'rate_30': feat['rate_30'],
        'recency_weighted': feat['recency_weighted'],
        'active_ratio_90': feat['active_ratio_90'],
        'avg_days_between': feat['avg_days_between'],
        'cv_days_between': feat['cv_days_between'],
        'expected_deliveries_in_horizon': expected_deliveries_in_horizon,
        'delivery_overdue_ratio': delivery_overdue_ratio,
    }
    
    feature_vector = pd.DataFrame([feature_dict])[feature_cols]
    
    # Two-stage prediction
    reg_pred = max(0, model.predict(feature_vector)[0])
    prob_pos = clf.predict_proba(feature_vector)[:, 1][0]
    pred = reg_pred * prob_pos
    
    # Guardrails (ORIGINAL FROM BEST MODEL)
    days_inactive = feat['days_since_last']
    total_365 = feat['total_weight_365d']
    cap_upper = (total_365 / 365.0) * horizon * 1.5
    
    if days_inactive > 365:
        pred = 0.0
    elif 180 < days_inactive <= 365:
        cold_cap = 0.08 * total_365
        pred = min(pred, cold_cap)
    
    pred = max(0.0, min(pred, cap_upper))
    
    predictions.append({'ID': row['ID'], 'predicted_weight': pred})
    
    if (idx + 1) % 5000 == 0:
        print(f"Processed {idx + 1}/{len(prediction_mapping)}...")

predictions_df = pd.DataFrame(predictions)

print("\n[7] PREDICTION STATISTICS")
print("-"*80)
print(f"Predictions mean: {predictions_df['predicted_weight'].mean():,.0f} kg")
print("\nPrediction statistics:")
print(predictions_df['predicted_weight'].describe())
print(f"Predictions > 0: {(predictions_df['predicted_weight'] > 0).sum()}")

# ============================================================================
# CREATE SUBMISSION
# ============================================================================
print("\n[8] CREATING SUBMISSION")
print("-"*80)

submission = sample_submission.copy()
submission['predicted_weight'] = predictions_df['predicted_weight'].values
submission.to_csv('STEP8_TIMING_FEATURES.csv', index=False)
print("✅ Saved to 'STEP8_TIMING_FEATURES.csv'")

print("\n" + "="*80)
print("COMPLETE - STEP 8: TIMING FEATURES ADDED")
print("="*80)
print("\nChanges from BEST MODEL (6236):")
print("  ✅ Added 4 timing features:")
print("     - avg_days_between")
print("     - cv_days_between")
print("     - expected_deliveries_in_horizon")
print("     - delivery_overdue_ratio")
print("  ✅ Alpha=0.1 (same as best model)")
print("  ✅ Kept original guardrails (same as best model)")
print("  ✅ Kept two-stage architecture (same as best model)")
print("  ✅ Total features: 19 (was 15)")
print("\n  Expected: +10 to +30 points from timing-aware features")
print("="*80)

STEP 9: SAFE TIMING FEATURES ONLY

[1] DATA CLEANING
Clean receivals: 122383

[2] PRE-COMPUTING TIMING STATISTICS
--------------------------------------------------------------------------------
Computed timing stats for 203 rm_ids
RM_IDs with computable intervals: 174

[3] CREATING TRAINING DATA WITH SAFE TIMING FEATURES
--------------------------------------------------------------------------------
Using 11 training dates x 5 horizons
Active rm_ids in 2024: 60
Processing date 1/11: 2024-01-01...
Processing date 2/11: 2024-02-01...
Processing date 3/11: 2024-03-01...
Processing date 4/11: 2024-04-01...
Processing date 5/11: 2024-05-01...
Processing date 6/11: 2024-06-01...
Processing date 7/11: 2024-07-01...
Processing date 8/11: 2024-08-01...
Processing date 9/11: 2024-09-01...
Processing date 10/11: 2024-10-01...
Processing date 11/11: 2024-11-01...

Generated 2725 training samples
Samples with target > 0: 1816 (66.6%)

[4] TIME-BASED TRAIN/VAL SPLIT
-------------------------------